In [ ]:
import pystac_client
import pystac
import odc.stac
import geopandas
import xarray 
import rioxarray
import planetary_computer
import pathlib
import pandas
import numpy
import folium
import dask.distributed
import branca.element, branca.colormap # Remove whitespace around small folium map

module_path = pathlib.Path.cwd() / '..' / 'scripts'
import sys
if str(module_path) not in sys.path:
    sys.path.append(str(module_path))
import utils

%load_ext autoreload
%autoreload 2


# Ensure geometries setup

In [ ]:
tiles_south_island = utils.create_tiles_south_island(distance_offshore = 3_000, tile_length = 10_000)
tiles_chatham_island = utils.create_tiles_chatham_island(distance_offshore = 3_000, tile_length = 10_000)
tiles_north_island = utils.create_tiles_north_island(distance_offshore = 3_000, tile_length = 10_000)
test_sites = utils.create_test_sites(distance_offshore = 3_000)
land = geopandas.read_file(utils.DATA_PATH / "vectors" / f"main_islands.gpkg")

# Set up - call only once each session

In [ ]:
client = dask.distributed.Client()
odc.stac.configure_rio(cloud_defaults=True, client=client)
display(client)

odc.stac.configure_rio(cloud_defaults=True, aws={"aws_unsigned": True})
catalogue = {"url": "https://planetarycomputer.microsoft.com/api/stac/v1",
             "collections": {"sentinel": "sentinel-2-l2a", "dem": "cop-dem-glo-30"}}
client = pystac_client.Client.open(catalogue["url"], modifier=planetary_computer.sign_inplace) 

# Site of interest - change date, site name and thresholds

In [ ]:
date_YYMM = "2024-01-10" # "2016-09-19" 
site_name = "akaroa" # 'rakiora', 'waikouaiti', 'chatham', 'akaroa', 'matau', 'wellington'
thresholds = {"min_ndvi": 0.03, "max_ndvi": 0.7, "max_ndwi": 0.1, "min_ndvri": 0.03, "max_ndwi2": -0.2,}

# Generate results

In [ ]:
site_index = (test_sites["name"] == site_name).idxmax()
site_bbox = test_sites.to_crs(utils.CRS_WSG).loc[site_index].geometry.bounds

output_path = utils.DATA_PATH / "rasters" / "test_sites" / site_name
output_path.mkdir(parents=True, exist_ok=True)

data = utils.kelp_single_day(client, date_YYMM, roi=test_sites.loc[[site_index]], thresholds=thresholds)
data.to_netcdf(output_path / f'{date_YYMM}.nc', format="NETCDF4", engine="netcdf4")

Load back in to avoid memory errors and create polygons defining kelp and non kelp areas

In [ ]:
data = rioxarray.rioxarray.open_rasterio(output_path / f'{date_YYMM}.nc', masked=True)#.squeeze("band", drop=True)
buffer = 3
kelp_polygons = utils.polygon_from_raster(data["kelp"])
kelp_polygons_buffered = kelp_polygons.buffer(-utils.RASTER_DEFAULTS["resolution"] * buffer).buffer(utils.RASTER_DEFAULTS["resolution"] * buffer)
ocean_buffered = test_sites.loc[[site_index]].to_crs(kelp_polygons.crs)
ocean_buffered = ocean_buffered.overlay(kelp_polygons, how="difference")
ocean_buffered = ocean_buffered.buffer(-utils.RASTER_DEFAULTS["resolution"] * buffer)

Saves out histograms in the same folder as the data of the ocean and kelp regions, and update the plots

In [ ]:
utils.plot_hists_single_day(data, date_YYMM, ocean_buffered, output_path)

In [ ]:
# Put it all on a single interactive map
center = [(site_bbox[1] + site_bbox[3]) / 2, (site_bbox[0] + site_bbox[2]) / 2]  # Assuming min_lon and max_lon are defined
m = folium.Map(location=center, zoom_start=13)

# RGB
rgb_dict = { "Satellite RBG": ["red", "green", "blue"], "False NIR color": ["nir", "green", "blue"],
        "False rededge-2 colour": ["rededge - Band 6 - Vegetation red edge 2", "green", "blue"], "False rededge-3 colour": ["rededge - Band 7 - Vegetation red edge 3", "green", "blue"] }

for title, names in rgb_dict.items():
    bands = utils.get_band_names_from_common(names)
    rgb = utils.normalise_rgb(data, bands)
    rgb.odc.to_rgba(bands=bands, vmin=0, vmax=1).odc.add_to(map=m, name=title)

# Categorical for the predicted classes and for the training data
data["kelp"].odc.add_to(m, name="Kelp", opacity=0.75, cmap="inferno", vmin=0, vmax=1)
data["ndvi"].odc.add_to(m, name="ndvi", opacity=0.75, cmap="inferno", vmin=0, vmax=1)
data["ndwi"].odc.add_to(map=m, name="ndwi", opacity=0.75, cmap="inferno", vmin=0, vmax=1)
data["ndvri"].odc.add_to(map=m, name="ndvri", opacity=0.75, cmap="inferno", vmin=0, vmax=1)
data["ndwi2"].odc.add_to(map=m, name="ndwi2", opacity=0.75, cmap="inferno", vmin=0, vmax=1)

colormap = branca.colormap.linear.inferno.scale(0, 1)
colormap.caption = 'Kelp Index'
colormap.add_to(m)

kelp_polygons.explore(m=m, color="magenta", style_kwds={"fillOpacity": 0}, name="kelp polygon")
kelp_polygons_buffered.explore(m=m, color="red", style_kwds={"fillOpacity": 0}, name="kelp buffered dialated polygon")
ocean_buffered.explore(m=m, color="blue", style_kwds={"fillOpacity": 0}, name="ocean dialated polygon")

# Layer control
folium.LayerControl().add_to(m)
import folium.plugins
folium.plugins.MousePosition(position="topright",).add_to(m)

m

In [ ]:
utils.SENTINEL_2B_BAND_INFO